# Trading Strategy Analysis

This notebook develops and evaluates trading strategies based on our model predictions.

## Contents
1. Setup and Data Loading
2. Basic Trading Strategy Development
3. Position Sizing and Risk Management
4. Strategy Performance Evaluation
5. Portfolio Simulation
6. Strategy Optimization


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import our custom modules
from src.training.model import CryptoPredictor
from src.visualization.visualizer import CryptoVisualizer

# Set random seed for reproducibility
np.random.seed(42)


## 1. Setup and Data Loading


In [ ]:
# Load model and predictions
model = CryptoPredictor.load('models/best_model.h5')

# Load historical data and predictions
historical_data = pd.read_csv('data/processed/test_data.csv',
                              index_col='timestamp',
                              parse_dates=True)

predictions = pd.read_csv('results/predictions.csv',
                          index_col='timestamp',
                          parse_dates=True)

print("Data loaded successfully")
print(f"Historical data shape: {historical_data.shape}")
print(f"Predictions shape: {predictions.shape}")


## 2. Basic Trading Strategy Development


In [ ]:
class TradingStrategy:
    """Basic trading strategy implementation"""
    
    def __init__(self, prediction_threshold=0.01, stop_loss=0.02, take_profit=0.03):
        self.prediction_threshold = prediction_threshold
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        
    def generate_signals(self, predictions, actual_prices):
        """Generate trading signals based on predictions"""
        signals = pd.DataFrame(index=predictions.index)
        
        # Calculate predicted returns
        pred_returns = (predictions - actual_prices) / actual_prices
        
        # Generate signals
        signals['position'] = 0  # Default to no position
        signals.loc[pred_returns > self.prediction_threshold, 'position'] = 1  # Buy signal
        signals.loc[pred_returns < -self.prediction_threshold, 'position'] = -1  # Sell signal
        
        return signals
    
    def apply_risk_management(self, signals, prices):
        """Apply stop loss and take profit rules"""
        positions = signals['position'].copy()
        entry_price = None
        current_position = 0
        
        for i in range(1, len(positions)):
            if current_position == 0 and positions[i] != 0:
                # Enter new position
                current_position = positions[i]
                entry_price = prices[i]
            elif current_position != 0:
                # Check stop loss and take profit
                returns = (prices[i] - entry_price) / entry_price * current_position
                if returns <= -self.stop_loss or returns >= self.take_profit:
                    # Close position
                    current_position = 0
                    entry_price = None
            
            positions[i] = current_position
        
        signals['position'] = positions
        return signals

# Create and test basic strategy
strategy = TradingStrategy()
signals = strategy.generate_signals(predictions['predicted_price'],
                                     historical_data['close'])
signals = strategy.apply_risk_management(signals, historical_data['close'])

# Plot signals
fig = go.Figure()
fig.add_trace(go.Scatter(x=historical_data.index, y=historical_data['close'],
                         name='Price'))
fig.add_trace(go.Scatter(x=signals[signals['position'] == 1].index,
                         y=historical_data.loc[signals['position'] == 1, 'close'],
                         mode='markers',
                         name='Buy Signal',
                         marker=dict(color='green', size=10)))
fig.add_trace(go.Scatter(x=signals[signals['position'] == -1].index,
                         y=historical_data.loc[signals['position'] == -1, 'close'],
                         mode='markers',
                         name='Sell Signal',
                         marker=dict(color='red', size=10)))
fig.show()
